# **Personal Finance Advisor**
Develop a system that offers personalized financial advice using rules for budget management and investment suggestions.
# Get started with the Gemini API: Python



In [ ]:
# installation of the package
!pip install -q -U google-generativeai

In [ ]:
import textwrap
import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text)->Markdown:
    text : str = text.replace("•", "  *")
    return Markdown(textwrap.indent(text, "> ", predicate=lambda _: True))

In [ ]:
# Used to securely store your API key
from google.colab import userdata

### Setup your API key

Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in Google AI Studio.

<a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>

Note that depending on where you are located, you might have to [enable billing](https://ai.google.dev/gemini-api/docs/billing#enable-cloud-billing) since the free tier is not available in [EEA (including EU), the UK, and CH](https://ai.google.dev/gemini-api/docs/billing#is-Gemini-free-in-EEA-UK-CH)

In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `GEMINI_API_KEY`.

Once you have the API key, pass it to the SDK. You can do this in two ways:

* Put the key in the `GEMINI_API_KEY` environment variable (the SDK will automatically pick it up from there).
* Pass the key to `genai.configure(api_key=...)`

In [ ]:
# Or use `os.getenv('GEMINI_API_KEY')` to fetch an environment variable.
GEMINI_API_KEY : str = userdata.get("GEMINI_API_KEY")

genai.configure(api_key=GEMINI_API_KEY)

## List models

Now you're ready to call the Gemini API. Use `list_models` to see the available Gemini models:

* `gemini-1.5-flash`: optimized for multi-modal use-cases where speed and cost are important. This should be your go-to model.
* `gemini-1.5-pro`: optimized for high intelligence tasks, the most powerful Gemini model

In [ ]:
list(genai.list_models())

In [ ]:
for m in genai.list_models():
    if "generateContent" in m.supported_generation_methods:
        print(m.name)

## Generate text from text inputs

Always start with the 'gemini-1.5-flash' model. It should be sufficient for most of your tasks:

In [ ]:
from google.generativeai import GenerativeModel

model : GenerativeModel = genai.GenerativeModel("gemini-1.5-flash")

In [ ]:
%%time
from google.generativeai.types.generation_types import GenerateContentResponse
# New Function: Handle Finance & Non-Finance Queries
def handle_user_query(user_query: str) -> str:
    """
    Process the user's query and determine the appropriate response based on its relevance to finance.

    Args:
        user_query (str): The user's input query.

    Returns:
        str: Response text from the bot.
    """
    chat = model.start_chat(
        history=[
            {
                "role": "model",
                "parts": (
                    "You are a Personal Finance Assistant bot. You are designed exclusively "
                    "to provide personalized financial advice, including budget management, "
                    "investment suggestions, and other finance-related queries. If the user "
                    "asks about topics unrelated to finance, kindly apologize and inform them "
                    "that you are solely a finance assistant."
                )
            },
        ]
    )

    response = chat.send_message(user_query)
    return response.text

In [ ]:
%%time
finance_query = input("User query: ")
advice = handle_user_query(finance_query)
display(to_markdown(advice))
# My monthly budget is 190000 PKR, and my expenses are higher. How can I save money?

In [ ]:
# Example non-finance query
non_finance_query = "What is the weather today?"
non_finance_response = handle_user_query(non_finance_query)
display(to_markdown(non_finance_response))